In [ ]:
import os

directory = "./assets/2002_4"

if not directory:
    directory = input("Input raw dataset directory:")

files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

count = 1
if files:
    curr = files[0].split("_")[0]
else:
    print("No files found in the directory.")
    exit()

for file in files:
    file_split = file.split("_")
    if file_split[0] != curr:
        curr = file_split[0]
        count += 1
    
    new_file_name = "_".join(file_split[1:])
    new_file_name = str(count).zfill(3) + "_" + new_file_name
    
    old_path = os.path.join(directory, file)
    new_path = os.path.join(directory, new_file_name)
    
    os.rename(old_path, new_path)
    print(f"Successfully changed file name from ({file}) to ({new_file_name})")

In [ ]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def skeletonize(img):
    """Alternative skeletonization implementation without ximgproc"""
    skel = np.zeros(img.shape, np.uint8)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    while True:
        open_img = cv2.morphologyEx(img, cv2.MORPH_OPEN, element)
        temp = cv2.subtract(img, open_img)
        eroded = cv2.erode(img, element)
        skel = cv2.bitwise_or(skel, temp)
        img = eroded.copy()
        if cv2.countNonZero(img) == 0:
            break
    return skel

def process_fingerprint(image_path, output_path, target_size=None, upscale_factor=1.0):
    # Read image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    # Resize if needed
    if target_size:
        image = cv2.resize(image, target_size, interpolation=cv2.INTER_LANCZOS4)
    elif upscale_factor != 1.0:
        h, w = image.shape
        image = cv2.resize(image, (int(w*upscale_factor), int(h*upscale_factor)), 
                         interpolation=cv2.INTER_LANCZOS4)
    
    # 1. Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(image)
    
    # 2. Noise reduction
    denoised = cv2.bilateralFilter(enhanced, 9, 75, 75)
    
    # 3. Adaptive thresholding
    thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                cv2.THRESH_BINARY_INV, 21, 7)
    
    # 4. Morphological operations
    kernel = np.ones((3,3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # 5. Skeletonization (using alternative method)
    skeleton = skeletonize(morph)
    
    # 6. Final inversion and saving
    result = cv2.bitwise_not(skeleton)
    cv2.imwrite(output_path, result)

def batch_process_fingerprints(input_folder, output_folder, upscale=False):
    """
    Process all fingerprints in a folder
    """
    os.makedirs(output_folder, exist_ok=True)
    files = [f for f in os.listdir(input_folder) if f.lower().endswith('.tif')]
    
    for filename in tqdm(files, desc="Processing Fingerprints"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        try:
            if upscale:
                process_fingerprint(input_path, output_path, upscale_factor=2.0)
            else:
                process_fingerprint(input_path, output_path)
        except Exception as e:
            print(f"\nError processing {filename}: {str(e)}")
            continue

if __name__ == "__main__":
    INPUT_FOLDER = "assets"
    OUTPUT_FOLDER = "processed"
    UPSCALE_IMAGES = True  # Set to True for 2x upscaling
    
    print("Starting fingerprint processing...")
    batch_process_fingerprints(INPUT_FOLDER, OUTPUT_FOLDER, upscale=UPSCALE_IMAGES)
    print("\nProcessing completed successfully!")

In [ ]:
import os
import shutil
from tqdm import tqdm

# Define paths
dataset_folder = "processed"
train_folder = "dataset/train"
test_folder = "dataset/test"

# Create train and test folders
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get all image files
all_files = [f for f in os.listdir(dataset_folder) if f.endswith('.tif')]  # Only get .tif files

# Separate files: _1.tif for test, others for train
test_files = [f for f in all_files if f.endswith("_1.tif")]
train_files = [f for f in all_files]  # Exclude test files from train

print(f"Found {len(all_files)} total fingerprint images")
print(f"- Training images: {len(train_files)}")
print(f"- Test images: {len(test_files)}")

# Function to copy files
def copy_files(file_list, destination_folder):
    for file in tqdm(file_list, desc=f"Copying to {os.path.basename(destination_folder)}"):
        src = os.path.join(dataset_folder, file)
        dest = os.path.join(destination_folder, file)
        shutil.copy2(src, dest)

# Copy files to respective folders
copy_files(train_files, train_folder)
copy_files(test_files, test_folder)

print("\nDataset organization completed:")
print(f"- Training set: {len(train_files)} images (copied to {train_folder})")
print(f"- Test set: {len(test_files)} images (copied to {test_folder})")
print(f"Original files remain intact in {dataset_folder}")

In [ ]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm

# Path configuration
test_dir = os.path.join("dataset", "test")

def apply_block_damage(image, block_size=70, num_blocks=10):
    """Apply rectangular block damage to fingerprint image with WHITE blocks"""
    damaged = np.copy(image)
    height, width = image.shape
    
    for _ in range(num_blocks):
        x = random.randint(0, width - block_size)
        y = random.randint(0, height - block_size)
        damaged[y:y+block_size, x:x+block_size] = 255  # White blocks (changed from 0 to 255)
    return damaged

# Get all test images
test_images = [f for f in os.listdir(test_dir) if f.endswith("_1.tif")]
print(f"Found {len(test_images)} test images to process")

# Process all images with block damage (overwriting originals)
for filename in tqdm(test_images, desc="Applying block damage"):
    image_path = os.path.join(test_dir, filename)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if image is not None:
        # Apply block damage and overwrite original
        damaged_image = apply_block_damage(image)
        cv2.imwrite(image_path, damaged_image)

print(f"\nSuccessfully applied block damage to {len(test_images)} images")
print(f"Original images in {test_dir} have been overwritten")

<!-- NOT FINAL -->

In [64]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Lambda, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Konfigurasi path
train_dir = "./dataset/train/"
test_dir = "./dataset/test/"

# Parameter
IMG_SIZE = (600, 600)  # Pertahankan resolusi tinggi
BATCH_SIZE = 8
EPOCHS = 30
NUM_CLASSES = 80  # 80 individu berbeda

In [65]:
# Load semua path gambar dan label
def load_data_paths(directory):
    image_paths = []
    labels = []
    
    for filename in os.listdir(directory):
        if filename.endswith('.tif'):
            # Format nama file: 'ID_XXX.tif' (contoh: '001_1.tif')
            class_id = filename.split('_')[0]  
            image_paths.append(os.path.join(directory, filename))
            labels.append(int(class_id) - 1)  # Konversi ke indeks 0-based
    
    return image_paths, labels

# Load data train dan test
train_paths, train_labels = load_data_paths(train_dir)
test_paths, test_labels = load_data_paths(test_dir)

sample_img = cv2.imread(train_paths[0], cv2.IMREAD_GRAYSCALE)
print(f"\nFirst image size: {sample_img.shape}")  # Should be (height, width)


First image size: (600, 600)


In [66]:
def preprocess_image(img_path, target_size=(600,600)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, target_size)
    img_rgb = np.stack([img]*3, axis=-1)
    return img_rgb / 255.0

In [71]:
class FingerprintDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size=BATCH_SIZE, img_size=IMG_SIZE, augment=False):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.augment = augment
        
    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))  # Fix: Calculate based on paths
    
    def __getitem__(self, idx):
        batch_paths = self.image_paths[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_labels = self.labels[idx*self.batch_size:(idx+1)*self.batch_size]
        
        X = []
        y = []
        
        for path, label in zip(batch_paths, batch_labels):
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, self.img_size)  # Force resize
            img_rgb = np.stack([img]*3, axis=-1)
            img_normalized = img_rgb / 255.0
            X.append(img_normalized)
            y.append(label)
        
        return np.array(X), tf.keras.utils.to_categorical(np.array(y), num_classes=NUM_CLASSES)

# Create generators
train_generator = FingerprintDataGenerator(train_paths, train_labels, augment=True)
test_generator = FingerprintDataGenerator(test_paths, test_labels, augment=False)

In [ ]:
def build_vgg16_600x600(num_classes):
    # Input layer for 600x600 RGB
    input_tensor = Input(shape=(600, 600, 3))
    
    # Load VGG16 without FC layers
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_tensor=input_tensor
    )
    
    # Freeze all VGG layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Custom head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model


model = build_vgg16_600x600(NUM_CLASSES)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    tf.keras.callbacks.ModelCheckpoint(
        'vgg16_600x600_best.h5',
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-7
    )
]

# 9. Training
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=test_generator,
    validation_steps=len(test_generator),
    epochs=EPOCHS,
    callbacks=callbacks
)

# 10. Evaluation
best_model = tf.keras.models.load_model('vgg16_600x600_best.h5')
test_loss, test_acc = best_model.evaluate(test_generator)
print(f'\nTest Accuracy: {test_acc*100:.2f}%')

Epoch 1/30


C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_69']
Received: inputs=Tensor(shape=(None, 600, 600, 3))
  warnings.warn(msg)


71/80 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.0172 - loss: 4.5610

In [ ]:
# Prediksi label
y_pred = np.argmax(best_model.predict(test_generator), axis=1)
y_true = test_generator.labels[:len(test_generator.image_paths)]

# Classification report
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

# Confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(20, 20))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.show()